這邊跟前面比較 利用改變訓練資料 使得流失跟非流失比率調整1:1 強化對流失的判斷 看看結果是否增加

最後選擇高分模型再做參數挑整 找出更適合模型

再由最後的測試資料驗證我們的模型是否跟訓練時目標準確度一樣

In [1]:
import numpy as np
import pandas as pd

In [2]:
kkboxdata2017=pd.read_csv('kkboxdata2017v2.csv',parse_dates=['registration_init_time'])
kkboxdata2017.head()

,Unnamed: 0,msno,city,bd,gender,registered_via,registration_init_time,"('num_25', 'mean')","('num_50', 'mean')","('num_75', 'mean')",...,"('total_secs', 'mean')",play_days,payment_method_id_mode,payment_plan_days_mode,plan_list_price_sum,actual_amount_paid_sum,is_auto_renew_sum,is_cancel_sum,transactions_times,is_churn
0,0,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,2011-09-14,5.258621,1.017241,0.827586,...,7163.224276,58,41,30,258,258,2,0,2,0
1,1,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,2011-09-18,1.384615,0.576923,0.538462,...,4270.220462,26,40,30,298,298,2,0,2,0
2,2,OoDwiKZM+ZGr9P3fRivavgOtglTEaNfWJO4KaJcTTts=,1,0,NaN,7,2011-09-18,1.523810,0.285714,0.285714,...,3052.110524,21,41,30,298,298,2,0,2,0
3,3,4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,28,female,9,2011-09-20,2.200000,0.400000,0.200000,...,3557.196400,5,36,30,360,360,2,0,2,0
4,4,Z6WIOK9vXy+e2XDBiioNAxuZ0ScXSU/Ebq4tUwqVSrE=,22,38,female,9,2011-09-29,1.800000,1.000000,0.700000,...,1662.418200,20,40,30,298,298,2,0,2,0


In [3]:
kkboxdata2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770929 entries, 0 to 770928
Data columns (total 23 columns):
Unnamed: 0                770929 non-null int64
msno                      770929 non-null object
city                      770929 non-null int64
bd                        770929 non-null int64
gender                    356099 non-null object
registered_via            770929 non-null int64
registration_init_time    770929 non-null datetime64[ns]
('num_25', 'mean')        770929 non-null float64
('num_50', 'mean')        770929 non-null float64
('num_75', 'mean')        770929 non-null float64
('num_985', 'mean')       770929 non-null float64
('num_100', 'mean')       770929 non-null float64
('num_unq', 'mean')       770929 non-null float64
('total_secs', 'mean')    770929 non-null float64
play_days                 770929 non-null int64
payment_method_id_mode    770929 non-null int64
payment_plan_days_mode    770929 non-null int64
plan_list_price_sum       770929 non-null int64


因為類別太多 先做虛擬變數

In [4]:
dummiesc=['city','gender','registered_via','payment_method_id_mode']
kkboxdata2017 =pd.get_dummies(kkboxdata2017, prefix_sep="__",columns=dummiesc,dummy_na=True,drop_first=True)
cols = list(kkboxdata2017)
cols.pop(cols.index('is_churn'))
cols.append('is_churn')
kkboxdata2017 = kkboxdata2017.loc[:,cols]
kkboxdata2017.columns

Index(['Unnamed: 0', 'msno', 'bd', 'registration_init_time',
       '('num_25', 'mean')', '('num_50', 'mean')', '('num_75', 'mean')',
       '('num_985', 'mean')', '('num_100', 'mean')', '('num_unq', 'mean')',
       '('total_secs', 'mean')', 'play_days', 'payment_plan_days_mode',
       'plan_list_price_sum', 'actual_amount_paid_sum', 'is_auto_renew_sum',
       'is_cancel_sum', 'transactions_times', 'city__3.0', 'city__4.0',
       'city__5.0', 'city__6.0', 'city__7.0', 'city__8.0', 'city__9.0',
       'city__10.0', 'city__11.0', 'city__12.0', 'city__13.0', 'city__14.0',
       'city__15.0', 'city__16.0', 'city__17.0', 'city__18.0', 'city__19.0',
       'city__20.0', 'city__21.0', 'city__22.0', 'city__nan', 'gender__male',
       'gender__nan', 'registered_via__4.0', 'registered_via__7.0',
       'registered_via__9.0', 'registered_via__13.0', 'registered_via__nan',
       'payment_method_id_mode__11.0', 'payment_method_id_mode__14.0',
       'payment_method_id_mode__16.0', 'payment_m

訓練資料 設計在20161201之前註冊會員

In [5]:
traindata=kkboxdata2017.query('registration_init_time<20161202')
traindata['is_churn'].value_counts()

0    703552
1     36157
Name: is_churn, dtype: int64

驗證資料 設計在20161202與20170101之間註冊會員

In [6]:
validdata=kkboxdata2017.query('20170101>registration_init_time>20161201')
validdata['is_churn'].value_counts()

0    13920
1     1016
Name: is_churn, dtype: int64

測試資料 設計在20170101之後註冊會員

In [7]:
testdata=kkboxdata2017.query('registration_init_time>20170101')
testdata['is_churn'].value_counts()

0    13971
1     1857
Name: is_churn, dtype: int64

看結果而言 流失比率是少數 如果要找出預測流失客群 屬於二元分類問題 另外要對資料不平衡的情況做處理


先塞選好資料需要用的留

刪除不用的欄位

In [8]:
traindata.drop(['Unnamed: 0', 'msno','registration_init_time',], axis=1,inplace =True)
validdata.drop(['Unnamed: 0', 'msno','registration_init_time',], axis=1,inplace =True)
testdata.drop(['Unnamed: 0', 'msno','registration_init_time',], axis=1,inplace =True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


訓練資料流失跟非流失各取30000做1:1訓練 再由正常的驗證資料 看看變化

In [9]:
traindata0 = pd.concat([traindata.query('is_churn==0').sample(30000),traindata.query('is_churn==1').sample(30000)])
from sklearn.utils import shuffle
traindata0 = shuffle(traindata0)
traindata0.head()

,bd,"('num_25', 'mean')","('num_50', 'mean')","('num_75', 'mean')","('num_985', 'mean')","('num_100', 'mean')","('num_unq', 'mean')","('total_secs', 'mean')",play_days,payment_plan_days_mode,...,payment_method_id_mode__34.0,payment_method_id_mode__35.0,payment_method_id_mode__36.0,payment_method_id_mode__37.0,payment_method_id_mode__38.0,payment_method_id_mode__39.0,payment_method_id_mode__40.0,payment_method_id_mode__41.0,payment_method_id_mode__nan,is_churn
159857,26,6.600000,3.000000,3.400000,2.800000,49.600000,14.600000,12542.559600,5,30,...,0,0,0,0,0,0,0,1,0,0
310481,0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,216.573000,1,30,...,0,0,0,0,0,0,0,1,0,0
411213,33,2.473684,0.473684,0.342105,0.236842,14.526316,12.710526,3638.219421,38,30,...,0,0,0,0,0,0,0,1,0,0
752539,0,2.857143,1.857143,1.000000,0.714286,2.571429,8.285714,966.461571,7,30,...,0,0,0,0,1,0,0,0,0,1
384406,0,1.705882,1.058824,0.352941,0.588235,4.764706,6.647059,1490.513118,17,30,...,0,0,1,0,0,0,0,0,0,1


類別要使用虛擬變數

這邊要做分類的機器學習 這邊我先把主流的分類器跑一次 之後再挑選幾個作參數調整

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import recall_score
from sklearn.ensemble import AdaBoostClassifier

In [11]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    #"rbf SVM": SVC(kernel='rbf',probability=True),
    #"linear SVM": SVC(kernel='linear',probability=True),
    #"poly SVM": SVC(kernel='poly',probability=True),
    #"sigmoid SVM": SVC(kernel='sigmoid',probability=True),
    "linear SVM sgd": SGDClassifier(loss='hinge'),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=100),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier(),
    "AdaBoost" : AdaBoostClassifier()
    #"QDA": QuadraticDiscriminantAnalysis(),
    #"Gaussian Process": GaussianProcessClassifier()
}

In [12]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.perf_counter()
        classifier.fit(X_train, Y_train)
        t_end = time.perf_counter()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        test_recall=recall_score(Y_test,classifier.predict(X_test))
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'recall': test_recall, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_r = [dict_models[key]['recall'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),5)), columns = ['classifier', 'train_score', 'test_score', 'recall', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'recall'] = training_r[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

再來利用 隨機森林幫助我們先挑選可能比較有用的變數

In [13]:
feat_labels = traindata.columns[:-1]

forest = RandomForestClassifier(n_estimators=100,
                                random_state=1)

forest.fit(traindata.iloc[:,:-1], traindata.iloc[:,-1])
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]
for f in range(traindata.shape[1]-1):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]],importances[indices[f]]))

 1) is_cancel_sum                  0.174265
 2) is_auto_renew_sum              0.089702
 3) play_days                      0.073530
 4) actual_amount_paid_sum         0.061961
 5) plan_list_price_sum            0.059900
 6) ('total_secs', 'mean')         0.048926
 7) ('num_unq', 'mean')            0.048279
 8) ('num_100', 'mean')            0.047850
 9) ('num_25', 'mean')             0.047415
10) transactions_times             0.046819
11) ('num_50', 'mean')             0.046249
12) ('num_985', 'mean')            0.046059
13) ('num_75', 'mean')             0.045344
14) payment_method_id_mode__38.0   0.032025
15) bd                             0.029572
16) payment_method_id_mode__39.0   0.012586
17) registered_via__9.0            0.006128
18) gender__male                   0.006017
19) payment_method_id_mode__41.0   0.005805
20) registered_via__4.0            0.005448
21) payment_method_id_mode__29.0   0.005264
22) registered_via__7.0            0.005039
23) payment_method_id_mode__36.0

挑選比重比較高的前21個再做訓練

In [14]:
dict_models = batch_classify(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1], validdata.loc[:,feat_labels[indices[0:21]]], validdata.iloc[:,-1], no_classifiers = 12)
display_dict_models(dict_models)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


trained Logistic Regression in 4.10 s
trained Nearest Neighbors in 0.12 s
trained linear SVM sgd in 0.08 s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


trained Gradient Boosting Classifier in 4.93 s
trained Decision Tree in 0.67 s
trained Random Forest in 9.21 s
trained Neural Net in 4.54 s
trained Naive Bayes in 0.04 s
trained XGBoost in 3.28 s
trained AdaBoost in 3.09 s


,classifier,train_score,test_score,recall,train_time
5,Random Forest,0.999983,0.898099,0.958661,9.211591
6,Neural Net,0.892167,0.896157,0.926181,4.544015
3,Gradient Boosting Classifier,0.929417,0.894885,0.962598,4.927943
8,XGBoost,0.928450,0.894416,0.963583,3.282818
9,AdaBoost,0.925450,0.893613,0.961614,3.091793
4,Decision Tree,1.000000,0.891470,0.874016,0.673076
0,Logistic Regression,0.920967,0.878548,0.971457,4.101097
7,Naive Bayes,0.887067,0.829339,0.962598,0.036974
1,Nearest Neighbors,0.780833,0.772630,0.761811,0.116582
2,linear SVM sgd,0.523183,0.146157,0.969488,0.083293


跟資料未調整比 recall明顯提升很多 linear SVM recall過低 不是我們想要的 其他模型表現皆有提升

針對上面的理想模型 再調參數 希望調出數值最佳的模型

RandomForestClassifier

In [32]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
scorer = make_scorer(recall_score, pos_label=1)
param_test1 = {
 'criterion':['gini', 'entropy'],
 'max_leaf_nodes': [2,5,10,100,150,200,500]
}
gs = GridSearchCV(estimator=RandomForestClassifier(),
                  param_grid = param_test1,scoring=scorer,n_jobs=-1,iid=False, cv=2)

gs = gs.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gs.best_score_)
print('best train params')
print(gs.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gs.predict(validdata.loc[:,feat_labels[indices[0:21]]])))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


best train recall
0.9595666666666667
best train params
{'criterion': 'entropy', 'max_leaf_nodes': 200}
valid recall
0.96751968503937


In [46]:
rff=RandomForestClassifier(criterion= 'entropy', max_leaf_nodes= 200)

GradientBoosting

In [33]:
%%time
param_grid = dict(learning_rate=[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],max_depth= [1,2,3],n_estimators=[100,150])
gs = GridSearchCV(estimator=GradientBoostingClassifier(),
                  param_grid=param_grid,
                  scoring=scorer,
                  cv=2,
                  n_jobs = -1)
gs = gs.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gs.best_score_)
print('best train params')
print(gs.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gs.predict(validdata.loc[:,feat_labels[indices[0:21]]])))

best train recall
0.9584666666666667
best train params
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
valid recall
0.9635826771653543
Wall time: 29.8 s


In [47]:
gbf=GradientBoostingClassifier({'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100})

XGBClassifier

In [ ]:
'''
param_test1 = {
 'max_depth':range(3,10),
 'min_child_weight':range(1,11)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(         learning_rate =0.1, n_estimators=140, max_depth=5,
min_child_weight=1, gamma=0, subsample=0.8,             colsample_bytree=0.8,
 objective= 'binary:logistic',  scale_pos_weight=1, seed=27), 
 param_grid = param_test1,     scoring=scorer,n_jobs=-1,iid=False, cv=2)
gsearch1=gsearch1.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearch1.best_score_)
print('best train params')
print(gsearch1.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearch1.predict(validdata.loc[:,feat_labels[indices[0:21]]])))
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring=scorer,n_jobs=-1,iid=False, cv=5)
gsearch3=gsearch3.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearch3.best_score_)
print('best train params')
print(gsearch3.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearch3.predict(validdata.loc[:,feat_labels[indices[0:21]]])))
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring=scorer,n_jobs=-1,iid=False, cv=5)
gsearch4=gsearch4.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearch4.best_score_)
print('best train params')
print(gsearch4.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearch4.predict(validdata.loc[:,feat_labels[indices[0:21]]])))
param_test7 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.6,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring=scorer,n_jobs=-1,iid=False, cv=5)
gsearch4=gsearch4.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearch4.best_score_)
print('best train params')
print(gsearch4.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearch4.predict(validdata.loc[:,feat_labels[indices[0:21]]])))
param_test7 = {
 'reg_alpha':np.arange(0.01,0.05,0.01)
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.6,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring=scorer,n_jobs=-1,iid=False, cv=5)
gsearch4=gsearch4.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearch4.best_score_)
print('best train params')
print(gsearch4.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearch4.predict(validdata.loc[:,feat_labels[indices[0:21]]])))
param_test8 = {
 'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.5]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.6,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test8, scoring=scorer,n_jobs=-1,iid=False, cv=2)
gsearch5=gsearch5.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearch5.best_score_)
print('best train params')
print(gsearch5.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearch5.predict(validdata.loc[:,feat_labels[indices[0:21]]])))
XGBb=GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.6,reg_alpha=0.01,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27)              
gsearchb=gsearchb.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])

'''

In [35]:
%%time
param_testa = {
 'max_depth':range(3,10),
 'min_child_weight':range(1,11),
 'reg_alpha':np.arange(0.01,0.05,0.01),
 'learning_rate':np.arange(0.01,0.05,0.01),
}
gsearchb = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.6,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_testa, scoring=scorer,n_jobs=-1,iid=False, cv=2)
gsearchb=gsearchb.fit(traindata0.loc[:,feat_labels[indices[0:21]]], traindata0.iloc[:,-1])
print('best train recall')
print(gsearchb.best_score_)
print('best train params')
print(gsearchb.best_params_)
print('valid recall')
print(recall_score(validdata.iloc[:,-1],gsearchb.predict(validdata.loc[:,feat_labels[indices[0:21]]])))

best train recall
0.9611666666666667
best train params
{'learning_rate': 0.01, 'max_depth': 8, 'min_child_weight': 1, 'reg_alpha': 0.04}
valid recall
0.9724409448818898
Wall time: 22min 17s


In [58]:
 xgbf=XGBClassifier(learning_rate =0.01, n_estimators=177, max_depth=8,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.6,reg_alpha=0.04,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27)

這邊完成我們理想模型 參數要調更細更準的話必須花費很多時間也不能保證測試可以相對提高只會有些微差距

最後再拿最新的資料測試 做最後驗證 這邊假設是我們用舊的資料想要預測新的資料是否跟我們模型想的一樣

先取測試時間之前的檔案再抽樣做成訓練資料

In [44]:
trainf = pd.concat([traindata,validdata])
trainf = pd.concat([trainf.query('is_churn==0').sample(30000),trainf.query('is_churn==1').sample(30000)])
trainf = shuffle(trainf)
dict_modelf = batch_classify(trainf.loc[:,feat_labels[indices[0:21]]], trainf.iloc[:,-1], testdata.loc[:,feat_labels[indices[0:21]]], testdata.iloc[:,-1], no_classifiers = 12)
display_dict_models(dict_modelf)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


trained Logistic Regression in 5.12 s
trained Nearest Neighbors in 0.12 s
trained linear SVM sgd in 0.06 s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


trained Gradient Boosting Classifier in 4.71 s
trained Decision Tree in 0.68 s
trained Random Forest in 9.04 s
trained Neural Net in 4.97 s
trained Naive Bayes in 0.04 s
trained XGBoost in 3.20 s
trained AdaBoost in 2.99 s


,classifier,train_score,test_score,recall,train_time
5,Random Forest,0.999967,0.885203,0.967690,9.039227
9,AdaBoost,0.923933,0.875284,0.980075,2.989527
3,Gradient Boosting Classifier,0.928050,0.874273,0.977383,4.705279
8,XGBoost,0.927067,0.874084,0.980075,3.196426
4,Decision Tree,1.000000,0.873642,0.901454,0.678212
2,linear SVM sgd,0.521100,0.871367,0.140549,0.064770
0,Logistic Regression,0.918467,0.863912,0.986537,5.116437
7,Naive Bayes,0.884683,0.810020,0.976306,0.036625
1,Nearest Neighbors,0.783583,0.753285,0.853527,0.119116
6,Neural Net,0.762600,0.591168,0.996230,4.969629


這是未調參數的情況下 之前預測的前三模型(Random Forest,Gradient Boosting Classifier	,XGBoost)結果表現差不多
再測試分數不能太低的情況下 想要最高的recall

再來看調整過參數 再用新的測試

In [59]:
dict_classifiersf = {
    "Gradient Boosting Classifier": xgbf,
    "Random Forest": rff,
    "XGBoost": xgbf,
}

In [60]:
def batch_classifyf(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiersf.items())[:no_classifiers]:
        t_start = time.perf_counter()
        classifier.fit(X_train, Y_train)
        t_end = time.perf_counter()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        test_recall=recall_score(Y_test,classifier.predict(X_test))
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'recall': test_recall, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_modelsf(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_r = [dict_models[key]['recall'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),5)), columns = ['classifier', 'train_score', 'test_score', 'recall', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'recall'] = training_r[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [61]:
dict_modelf = batch_classifyf(trainf.loc[:,feat_labels[indices[0:21]]], trainf.iloc[:,-1], testdata.loc[:,feat_labels[indices[0:21]]], testdata.iloc[:,-1], no_classifiers = 5)
display_dict_modelsf(dict_modelf)

trained Gradient Boosting Classifier in 11.23 s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


trained Random Forest in 0.77 s
trained XGBoost in 11.48 s


,classifier,train_score,test_score,recall,train_time
0,Gradient Boosting Classifier,0.933500,0.877053,0.979537,11.229288
2,XGBoost,0.933500,0.877053,0.979537,11.479489
1,Random Forest,0.929817,0.867766,0.983306,0.765590


調整後 recall 除了XGBoost 其他皆有提升一點 推測XGBoost調整過多參數會有overfit情形 不過結果依然優秀

這邊結論是由舊的會員資料做模型預測新的資料 給我們想要的目標(recall) 經過測試 跟之前訓練的資料差不多

可以藉由訓練挑選出適合的模型